# Minimal dummy example with zerobnl

We consider the following two nodes graph as a minimal dummy example of two nodes exchanging a variable and randomly add or substract a given value from it:

<img src="./model.png" alt="Model" style="width: 500px;"/>

The above graph can be created using zerobnl as follow:

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

`RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88`

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

sim.create_meta_model("MetaBase", [("a", "unit")], [("b", "unit")])
sim.create_environment("EnvBase", "wrapper_base.py", "Dockerfile_base")

sim.add_node("Base0", "MetaBase", "EnvBase", init_values={"c": 0.50}, files=["empty_file_for_example_purpose.txt"])
sim.add_node("Base1", "MetaBase", "EnvBase", init_values={"c": 0.25})

sim.add_link("Base0", "b", "Base1", "a")
sim.add_link("Base1", "b", "Base0", "a")

sim.create_sequence([["Base0"], ["Base1"]])
sim.set_time_unit("seconds")
sim.create_steps([15] * 4 * 60)

This simulation is based on `wrapper_base.py`, you can use to it as template to create your own. 

In [3]:
sim.run()

INFO :: Simulation finished in 0 min and 8 sec


The first time you run this cell, it can take a long time, all the containers have to be build from scratch, once it's done it will run much faster.

If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,OUT,Base1,b
1,X,Base1,y
2,X,Base0,y
3,OUT,Base0,b


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - b


OUT||Base1||b
2000-01-01 00:00:15   -0.75
2000-01-01 00:00:30   -0.50
2000-01-01 00:00:45   -1.25
2000-01-01 00:01:00   -1.50
2000-01-01 00:01:15   -1.25
                       ... 
2000-01-01 00:59:00    0.50
2000-01-01 00:59:15    1.00
2000-01-01 00:59:30    1.25
2000-01-01 00:59:45    1.00
2000-01-01 01:00:00    1.00
Length: 240, dtype: float64


In [6]:
for key, value in sim.get_results_by_pattern("X*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - y


X||Base1||y
2000-01-01 00:00:15   -1.0
2000-01-01 00:00:30   -1.0
2000-01-01 00:00:45   -1.0
2000-01-01 00:01:00   -1.0
2000-01-01 00:01:15   -1.0
                      ... 
2000-01-01 00:59:00    1.0
2000-01-01 00:59:15    0.0
2000-01-01 00:59:30   -1.0
2000-01-01 00:59:45    1.0
2000-01-01 01:00:00    0.0
Length: 240, dtype: float64
